In [1]:
import re
from bs4 import BeautifulSoup

### Split warc file to docs

In [42]:
# ref : https://stackoverflow.com/questions/60269904/split-text-file-after-specific-line-in-python
SECTION_START = re.compile(r'<!DOCTYPE html')
SECTION_END = re.compile(r'</html>')

def split_docs_iter(stream):
    def inner(stream):
        # Yields each line until an end marker is found (or EOF)
        for line in stream:
            if line and not SECTION_END.match(line):
                yield line
                continue
            break

    # Find a start marker, then break off into a nested iterator
    for line in stream:
        if line:
            if SECTION_START.match(line):
                yield inner(stream)
            continue
        break

In [44]:
filename = "03.warc"

# split docs
with open(filename, 'r', encoding="ISO-8859-1") as fh_in:
    for (i, nested_iter) in enumerate(split_docs_iter(fh_in)):
        with open('./docs/docID_{:05d}'.format(i), 'w', encoding='UTF-8') as fh_out:
            for line in nested_iter:
                fh_out.write(line)

### Parse html and get text in <body> tag 

In [41]:
from os import listdir
from os.path import isfile, join

DOCS_PATH = 'docs'
PROCESSED_DOCS_PATH = 'processed_docs'

files = [f for f in listdir(DOCS_PATH) if isfile(join(DOCS_PATH, f))]

for file in files:
    with open(f"{DOCS_PATH}/{file}", 'r', encoding="ISO-8859-1") as f:
        soup = BeautifulSoup(f, "html.parser")

        try:
            # get text in <body>
            body_text = soup.find('body').getText()
            # Home\nHi -> Home Hi
            concatenated_body_text = " ".join(body_text.split())
            # A -> a, additional character -> ""
            processed_concatenated_body_text = re.sub(r"[^A-Za-z0-9]+", ' ', concatenated_body_text).lower()

            with open(f"{PROCESSED_DOCS_PATH}/{file}_processed", mode="w", encoding="utf-8", errors='strict', buffering=1) as f1:
                f1.write(processed_concatenated_body_text)
        # skip the docs which not have <body>
        except AttributeError:
            print(f"{file} don't have body")


docID_02492 don't have body
docID_05721 don't have body
docID_05722 don't have body
docID_05723 don't have body
docID_05724 don't have body
docID_05725 don't have body
docID_05726 don't have body
docID_05727 don't have body
docID_05728 don't have body
docID_05729 don't have body
docID_05792 don't have body


OSError: [Errno 22] Invalid argument: 'docs/docID_05842'

In [38]:
with open(f"./{DOCS_PATH}/docID_01861", 'r', encoding="ISO-8859-1") as f:
    soup = BeautifulSoup(f, "html.parser")

    # get text in <body>
    body_text = soup.find('body').getText()
    # Home\nHi -> Home Hi
    concatenated_body_text = " ".join(body_text.split())
    # A -> a, additional character -> ""
    processed_concatenated_body_text = re.sub(r"[^A-Za-z0-9]+", ' ', concatenated_body_text).lower()